In [1]:
%pip install openai numpy matplotlib pydantic --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
from dotenv import load_dotenv
import os
from openai import OpenAI

In [7]:
MODEL = "gpt-4.1-mini"

In [8]:
# Initialize the client
# Note: In a real application, you would use an environment variable or secure method
# to store your API key. This is just for demonstration.


load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

1. Basic Text Generation

In [9]:
prompt = "Write a one-sentence bedtime story about a unicorn."

response = client.responses.create(
    model=MODEL,
    input=prompt
)

print(f"Prompt: {prompt}")
print(f"Response: {response.output_text}")

# Show how to control generation with parameters
print("\nWith controlled parameters:")
response = client.responses.create(
    model=MODEL,
    input=prompt,
    temperature=0.7,  # Lower for more deterministic outputs
    top_p=0.9         # Nucleus sampling
)

print(f"Response (controlled): {response.output_text}")

Prompt: Write a one-sentence bedtime story about a unicorn.
Response: Under a shimmering moonlit sky, a gentle unicorn sprinkles stardust over a sleepy forest, filling every dream with magic and wonder.

With controlled parameters:
Response (controlled): Under a silver moon, a gentle unicorn with shimmering wings whispered dreams to the stars, filling the night with magic and peace.


2. Structured Outputs

In [10]:
import json

response = client.responses.create(
   model=MODEL,
    input=[
        {"role": "system", "content": "You are a UI generator AI. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "ui",
            "description": "Dynamically generated UI",
            "schema": {
                "type": "object",
                "properties": {
                    "type": {
                        "type": "string",
                        "description": "The type of the UI component",
                        "enum": ["div", "button", "header", "section", "field", "form"]
                    },
                    "label": {
                        "type": "string",
                        "description": "The label of the UI component, used for buttons or form fields"
                    },
                    "children": {
                        "type": "array",
                        "description": "Nested UI components",
                        "items": {"$ref": "#"}
                    },
                    "attributes": {
                        "type": "array",
                        "description": "Arbitrary attributes for the UI component, suitable for any element",
                        "items": {
                            "type": "object",
                            "properties": {
                              "name": {
                                  "type": "string",
                                  "description": "The name of the attribute, for example onClick or className"
                              },
                              "value": {
                                  "type": "string",
                                  "description": "The value of the attribute"
                              }
                          },
                          "required": ["name", "value"],
                          "additionalProperties": False
                      }
                    }
                },
                "required": ["type", "label", "children", "attributes"],
                "additionalProperties": False
            },
            "strict": True,
        },
    },
)

ui = json.loads(response.output_text)

In [11]:
ui

{'type': 'form',
 'label': '',
 'children': [{'type': 'section',
   'label': 'User Profile Form',
   'children': [{'type': 'field',
     'label': 'First Name',
     'children': [],
     'attributes': [{'name': 'type', 'value': 'text'},
      {'name': 'name', 'value': 'firstName'},
      {'name': 'placeholder', 'value': 'Enter your first name'}]},
    {'type': 'field',
     'label': 'Last Name',
     'children': [],
     'attributes': [{'name': 'type', 'value': 'text'},
      {'name': 'name', 'value': 'lastName'},
      {'name': 'placeholder', 'value': 'Enter your last name'}]},
    {'type': 'field',
     'label': 'Email',
     'children': [],
     'attributes': [{'name': 'type', 'value': 'email'},
      {'name': 'name', 'value': 'email'},
      {'name': 'placeholder', 'value': 'Enter your email'}]},
    {'type': 'field',
     'label': 'Password',
     'children': [],
     'attributes': [{'name': 'type', 'value': 'password'},
      {'name': 'name', 'value': 'password'},
      {'name': '

2.1 Structured Outputs with Pydantic models

In [12]:
from pydantic import BaseModel

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

steps=[Step(explanation='Start with the equation: 8x + 7 = -23.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to isolate the term with x on one side.', output='8x + 7 - 7 = -23 - 7, which simplifies to 8x = -30'), Step(explanation='Divide both sides of the equation by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction if possible.', output='x = -15 / 4')] final_answer='x = -15/4'


In [13]:
math_reasoning.parsed.steps[0]

Step(explanation='Start with the equation: 8x + 7 = -23.', output='8x + 7 = -23')

3. Multimodal Capabilities - Vision

In [14]:
# For notebook demonstration, we'll use a placeholder URL
image_url = "https://images.unsplash.com/photo-1579546929518-9e396f3cc809?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxleHBsb3JlLWZlZWR8MXx8fGVufDB8fHx8&w=1000&q=80"

response = client.responses.create(
    model=MODEL,
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": image_url,
            },
        ],
    }],
)

print(response.output_text)

This image features a colorful gradient background. The colors blend smoothly from warm tones like pink, orange, and peach on the top left to cooler tones like blue and light purple on the bottom right. There are no distinct objects or figures in the image; it is an abstract color gradient.


4. Audio Capabilities - Text to Speech

In [15]:
speech_file_path = "speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build something people love!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

4.1 Audio Capabilities - Speech to Text

In [16]:
audio_file= open("speech.mp3", "rb")

transcription = client.audio.transcriptions.create(
    model="gpt-4o-transcribe", 
    file=audio_file
)

print(transcription.text)

Today is a wonderful day to build something people love.


5. Function Calling

In [17]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)

In [18]:
# Extract the tool call and arguments
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
# Call the function
result = get_weather(args["latitude"], args["longitude"])

In [19]:
# Append the tool call and result to the input messages
input_messages.append(tool_call) # append model's function call message
input_messages.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1-mini",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in Paris today is 7.6°C. If you want more detailed weather information, please let me know!


--------------

In [ ]:
6. Reasoning Models

In [20]:
# response_o1 = client.responses.create(
#     model="o1",
#     input="Design an algorithm to find the shortest path in a graph."
# )

# print("o1 Response:")
# print(response_o1.output_text)

# Using o4-mini for faster reasoning
response_o4 = client.responses.create(
    model="o4-mini",
    input="Explain how to implement a hash table."
)

print("\no3-mini Response:")
print(response_o4.output_text)


o3-mini Response:
Here’s a step-by-step guide to implementing a basic hash table, covering both separate-chaining and open-addressing strategies, plus essential operations and resizing.

1. Core components  
   • Array of buckets of fixed length M  
   • A hash function h(key) → integer in [0…M–1]  
   • Collision-resolution strategy (separate chaining or open addressing)  

2. Choice of hash function  
   • For strings: polynomial rolling hash, e.g.  
     hash = (∑ key[i]·pⁱ) mod M  
   • For integers: simple modular reduction, e.g. h(k) = k mod M  

3. Strategy A: Separate Chaining  
   – Each bucket is a linked list (or dynamic array) of (key, value) pairs.  
   – Load factor α = N/M; keep α ≤ 1 or 2 for good performance.

   Pseudocode (assume buckets[0…M–1] each is initially empty list):
   ```
   function insert(key, value):
     i = h(key)
     for each (k,v) in buckets[i]:
       if k == key:
         v = value            # update existing
         return
     buckets[i].appe

In [21]:
## With reasoning models, you can view the reasoning process and the steps taken to arrive at the answer.
print("This is the reasoning configuration for o4-mini:", response_o4.reasoning)

This is the reasoning configuration for o4-mini: Reasoning(effort='medium', generate_summary=None, summary=None)


7. Embeddings

In [22]:
import numpy as np

# First, let's create embeddings for a set of words
words = [
    "king", "queen", "man", "woman", 
    "apple", "banana", "orange", "pear",
    "castle", "throne"
]

# Get embeddings for all words
response = client.embeddings.create(
    model="text-embedding-3-large",
    input=words,
    encoding_format="float"
)

# Extract the embeddings
embeddings = [data.embedding for data in response.data]

print(f"Embedding dimension: {len(embeddings[0])}")
print(f"Number of embeddings: {len(embeddings)}")

# Function to compute dot product between two vectors
def dot_product(vec1, vec2):
    return np.dot(vec1, vec2)

# Compute similarity matrix (dot products between all pairs)
similarity_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        similarity_matrix[i][j] = dot_product(embeddings[i], embeddings[j])

# Print similarity matrix with labels
print("\nSimilarity Matrix (Dot Products):")
print("          " + " ".join(f"{word:<8}" for word in words))
for i, word in enumerate(words):
    row_values = " ".join(f"{similarity_matrix[i][j]:.4f}  " for j in range(len(words)))
    print(f"{word:<10} {row_values}")

# Check specific relationships
king_queen_similarity = dot_product(embeddings[0], embeddings[1])
apple_banana_similarity = dot_product(embeddings[4], embeddings[5])

print("\nSpecific relationships:")
print(f"Similarity between 'king' and 'queen': {king_queen_similarity:.4f}")
print(f"Similarity between 'apple' and 'banana': {apple_banana_similarity:.4f}")
print(f"Similarity between 'king' and 'apple': {dot_product(embeddings[0], embeddings[4]):.4f}")

# We expect king/queen to be closer to each other than king/apple
# And apple/banana to be closer to each other than queen/banana

Embedding dimension: 3072
Number of embeddings: 10

Similarity Matrix (Dot Products):
          king     queen    man      woman    apple    banana   orange   pear     castle   throne  
king       1.0000   0.5552   0.4183   0.2938   0.3243   0.3305   0.2879   0.2802   0.3615   0.4027  
queen      0.5552   1.0000   0.3073   0.4133   0.3146   0.3191   0.2983   0.2995   0.2969   0.3353  
man        0.4183   0.3073   1.0000   0.5713   0.3098   0.3495   0.2973   0.2695   0.2998   0.2650  
woman      0.2938   0.4133   0.5713   1.0000   0.3199   0.2937   0.2784   0.2533   0.2449   0.2491  
apple      0.3243   0.3146   0.3098   0.3199   1.0000   0.4619   0.4588   0.4392   0.3002   0.2340  
banana     0.3305   0.3191   0.3495   0.2937   0.4619   1.0000   0.4579   0.3636   0.2777   0.2075  
orange     0.2879   0.2983   0.2973   0.2784   0.4588   0.4579   1.0000   0.3822   0.2848   0.2174  
pear       0.2802   0.2995   0.2695   0.2533   0.4392   0.3636   0.3822   1.0000   0.2788   0.1929  
castle